In [2]:
import pandas as pd

In [3]:
# Leitura dos dados
df = pd.read_csv("docs/teste.csv")

df

,op_padronizado,dt_ocorrencia,hr_ocorrencia,uf,regiao,latitude,longitude,cat_aeronave,fase_operacao,modelo_aeronave,peso_max_decolagem,numero_assentos,nome_fabricante,les_fatais_trip
0,AEROMAJ AVIACAO AGRICOLA LTDA,03/01/2020,21:15:00,SP,Sudeste,"-22,3689","-47,7875",S05,Decolagem,EMB-202A,1800,1.0,NEIVA,0
1,SAVANA AERO AGRICOLA,11/01/2020,11:30:00,PR,Sul,"-23,3875","-52,875",S05,Táxi,A188B,1905,1.0,CESSNA AIRCRAFT,0
2,SAVANA AERO AGRICOLA,11/01/2020,11:30:00,PR,Sul,"-23,3875","-52,875",S05,Táxi,EMB-201A,1800,1.0,NEIVA,0
3,AVANTE AVIACAO AGRICOLA LTDA,15/01/2020,09:30:00,RS,Sul,"-28,4775","-53,8922",S05,Manobra,AT-402B,3175,1.0,AIR TRACTOR,0
4,ALIANCA AVIACAO AGRICOLA LTDA,17/01/2020,15:30:00,MT,Centro-Oeste,"-16,2722","-53,8958",S05,Manobra,AT-402B,3175,1.0,AIR TRACTOR,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,VOLMAR DUDA,10/05/2024,10:00:00,RO,Norte,"-13,2706","-60,9583",TPP,Decolagem,EMB-202A,1800,1.0,EMBRAER,0
72,KONAGESKI SERVICOS AGROPECUÁRIOS AEROAGRÍCOLA ...,15/05/2024,19:00:00,MT,Centro-Oeste,"-13,8914","-56,9114",S05,Decolagem,AT-502B,3629,1.0,AIR TRACTOR,0
73,AGROPECUARIA TRES ESTRELAS LTDA-ME,04/06/2024,18:15:00,MT,Centro-Oeste,"-15,1206","-53,7306",TPP,Decolagem,EMB-202A,1800,1.0,EMBRAER,0
74,AGROPECUARIA TRES ESTRELAS LTDA-ME,04/06/2024,18:15:00,MT,Centro-Oeste,"-15,1206","-53,7306",TPP,Decolagem,EMB-202A,1800,1.0,EMBRAER,0


# Transformação dos dados

In [4]:
# Pegar informações da Tabela
# O interpretador só entende numeros, então precisa converter tudo para int, float ou binário
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   op_padronizado      76 non-null     object 
 1   dt_ocorrencia       76 non-null     object 
 2   hr_ocorrencia       76 non-null     object 
 3   uf                  76 non-null     object 
 4   regiao              76 non-null     object 
 5   latitude            75 non-null     object 
 6   longitude           75 non-null     object 
 7   cat_aeronave        76 non-null     object 
 8   fase_operacao       76 non-null     object 
 9   modelo_aeronave     76 non-null     object 
 10  peso_max_decolagem  76 non-null     int64  
 11  numero_assentos     75 non-null     float64
 12  nome_fabricante     76 non-null     object 
 13  les_fatais_trip     76 non-null     int64  
dtypes: float64(1), int64(2), object(11)
memory usage: 8.4+ KB


# Transformação de dados

In [5]:
# Aqui estou tentando mudar o tipo de manobra / estado para numeros na qual o interpretador consiga diferenciar
convercao_dados = {
    "avioes":{0:"Decolagem", 1:"Manobra", 2:"Taxi"},
    "estado:":{0:"SP",1:"RJ",2:"MG",3:"RO",4:"MT",5: "BA"}
}
# Salva as configurações
df = df.replace(convercao_dados)

In [6]:
# Deletar duplicatas da tabela
df = df.drop_duplicates()

# Remove linhas com os valores Nulos
# Pode ser interessante remover essa linha para verificar a precisão do código
df = df.dropna()

In [7]:
# Comando para verificar se há valores nulos na tabela
print(df.isnull().sum())

op_padronizado        0
dt_ocorrencia         0
hr_ocorrencia         0
uf                    0
regiao                0
latitude              0
longitude             0
cat_aeronave          0
fase_operacao         0
modelo_aeronave       0
peso_max_decolagem    0
numero_assentos       0
nome_fabricante       0
les_fatais_trip       0
dtype: int64


In [8]:
# Converte a latitude e Longitude em float
# Os dois tem , no meio. Então o INT não ia dar certo
df['latitude'] = df['latitude'].str.replace(',', '.').astype(float)
df['longitude'] = df['longitude'].str.replace(',', '.').astype(float)

In [11]:
# A biblioteca LabelEncoder do scikit-learn serve para transformar dados (textos ou rótulos) em números inteiros
from sklearn.preprocessing import LabelEncoder

# Lista das colunas categóricas que você quer transformar
colunas_categoricas = ['fase_operacao', 'cat_aeronave', 'regiao', 'uf', 'modelo_aeronave', 'nome_fabricante']

# Aplica LabelEncoder em cada coluna da lista
for col in colunas_categoricas:
    df[col] = LabelEncoder().fit_transform(df[col])


In [12]:
# Escolha as colunas que fazem sentido para prever acidentes
X = df[['latitude', 'longitude', 'peso_max_decolagem', 'numero_assentos', 'fase_operacao', 'cat_aeronave', 'regiao', 'uf']]
y = df['les_fatais_trip']

# X vai ser as colunas que queremover verificar no Y (Se teve acidentes fatais ou não)
# Então adiciona tudo as tabelas por enquanto, mas é adaptável conforme formos estudando

In [13]:

# Essa bibliteca importa uma função do scikit-learn na qual faz dois conjuntos, um treina o conjunto e outro testa se está correto
# Isso evita que o modelo não "decore" os dados, e sim aprenda
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [14]:
# Serve para criar uma arvore de decisão para classificação dos dados
from sklearn.tree import DecisionTreeClassifier

# Treina o modelo com base no X_train, Y_train e verifica as respostas corretas com  base no Y_test, Y_test
model = DecisionTreeClassifier()
model.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [18]:
# Importa funções que calculam métricas de avaliações
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Precisão:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Acurácia: 0.6153846153846154
Precisão: 0.5
Recall: 0.4
F1 Score: 0.4444444444444444
